In [1]:
# Install surya-ocr if needed
# !pip install surya-ocr pillow pymupdf matplotlib

In [ ]:
import os
from pathlib import Path
import fitz  # PyMuPDF
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# NEW: Import the predictor class instead of the old function
from surya.detection import DetectionPredictor

/Users/houmanrajabi/miniconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'batch_text_detection' from 'surya.detection' (/Users/houmanrajabi/miniconda3/envs/thesis/lib/python3.10/site-packages/surya/detection/__init__.py)

In [ ]:
# Configuration
PDF_PATH = "../data/raw/fomcprojtabl20230322.pdf"  # UPDATE THIS
OUTPUT_DIR = "surya_debug"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Load Surya models
print("Loading Surya models...")
predictor = DetectionPredictor()
print("Models loaded")

In [ ]:
def pdf_page_to_image(pdf_path, page_num):
    """Convert PDF page to PIL Image"""
    doc = fitz.open(pdf_path)
    page = doc[page_num]
    pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # 144 DPI
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    doc.close()
    return img

In [ ]:
def detect_and_visualize(pdf_path, page_num, output_dir):
    """Detect tables/figures and visualize bboxes"""
    
    # Convert page to image
    img = pdf_page_to_image(pdf_path, page_num)
    
    # NEW: Run detection using the class instance
    # Pass a list of images, returns a list of results
    predictions = predictor([img])
    result = predictions[0]
    
    # Visualize
    fig, ax = plt.subplots(1, 1, figsize=(15, 20))
    ax.imshow(img)
    
    # Draw bboxes
    # Note: result.bboxes is still the correct attribute to iterate over
    for bbox in result.bboxes:
        # The .bbox attribute usually contains [x1, y1, x2, y2]
        x1, y1, x2, y2 = bbox.bbox
        width = x2 - x1
        height = y2 - y1
        
        rect = patches.Rectangle(
            (x1, y1), width, height,
            linewidth=2,
            edgecolor='red',
            facecolor='none'
        )
        ax.add_patch(rect)
    
    ax.set_title(f"Page {page_num + 1} - Detected {len(result.bboxes)} regions", fontsize=16)
    ax.axis('off')
    
    # Save
    output_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Page {page_num + 1}: {len(result.bboxes)} regions detected")
    print(f"Saved to {output_path}")
    
    return result

In [ ]:
# Process all pages in PDF
doc = fitz.open(PDF_PATH)
num_pages = len(doc)
doc.close()

print(f"Processing {num_pages} pages from {Path(PDF_PATH).name}\n")

results = []
for page_num in range(num_pages):
    result = detect_and_visualize(PDF_PATH, page_num, OUTPUT_DIR)
    results.append(result)
    print()

In [ ]:
# Print bbox details
for page_num, result in enumerate(results):
    print(f"\nPage {page_num + 1}:")
    for i, bbox in enumerate(result.bboxes, 1):
        x1, y1, x2, y2 = bbox.bbox
        print(f"  Region {i}: [{x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f}] - Size: {x2-x1:.1f}x{y2-y1:.1f}")